In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy.external as sce
import seaborn as sns
from matplotlib.pyplot import rc_context

import random

In [29]:
# read in scanpy data

rna=sc.read('data_files/rna_analyzed.h5') # 12892 × 36691 gene expression only from data -- processed w/ sc.pp.normalize_total(rna), sc.pp.log1p(rna) 
## rna: adata.X and adata.layers should be normalized using default params

## rna.raw should have log transformed counts (log1p) from data.X

## rna.obs metadata table will have labels established from analysis done in 02 notebook

# original / raw data 12892 × 36692
data=sc.read('data_files/3data_raw.h5') # concatenated filtered counts for 3 donors; no normalization done

In [31]:
rna.obs

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,sample,leiden,Ig,IgL,IgH,sec
AAACCCAAGTACTGTC-1-0,3128,3128,21325.0,909.0,4.262603,0,1,G,kappa,G1,x
AAACCCAAGTGAATAC-1-0,1774,1774,6060.0,518.0,8.547854,0,3,M,kappa,M,x
AAACCCAGTCCACAGC-1-0,2914,2914,19771.0,792.0,4.005867,0,1,G,kappa,G1,hi
AAACCCATCACAGTGT-1-0,3122,3122,12608.0,757.0,6.004125,0,1,M,kappa,M,hi
AAACCCATCCCTCTCC-1-0,3021,3021,16684.0,247.0,1.480460,0,1,G,kappa,G1,hi
...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCGACTCCT-1-2,2467,2467,14874.0,309.0,2.077451,2,7,M,lambda,M,x
TTTGTTGAGGTACAAT-1-2,2159,2159,14240.0,170.0,1.193820,2,7,M,lambda,M,x
TTTGTTGAGTAACGAT-1-2,2665,2665,18230.0,674.0,3.697202,2,5,A,kappa,A,lo
TTTGTTGCACGTTGGC-1-2,2211,2211,19481.0,384.0,1.971151,2,5,A,kappa,A,lo


In [32]:
# add rna leiden clusters and processed metadata to raw data object
data.obs['leiden']=rna.obs['leiden']
data.obs['Ig']=rna.obs['Ig']
data.obs['IgL']=rna.obs['IgL']
data.obs['IgH']=rna.obs['IgH']
data.obs['sec']=rna.obs['sec']

In [33]:
data.obs

,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,sample,leiden,Ig,IgL,IgH,sec
AAACCCAAGTACTGTC-1-0,3128,3128,21325.0,909.0,4.262603,0,1,G,kappa,G1,x
AAACCCAAGTGAATAC-1-0,1774,1774,6060.0,518.0,8.547854,0,3,M,kappa,M,x
AAACCCAGTCCACAGC-1-0,2914,2914,19771.0,792.0,4.005867,0,1,G,kappa,G1,hi
AAACCCATCACAGTGT-1-0,3122,3122,12608.0,757.0,6.004125,0,1,M,kappa,M,hi
AAACCCATCCCTCTCC-1-0,3021,3021,16684.0,247.0,1.480460,0,1,G,kappa,G1,hi
...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCGACTCCT-1-2,2467,2467,14874.0,309.0,2.077451,2,7,M,lambda,M,x
TTTGTTGAGGTACAAT-1-2,2159,2159,14240.0,170.0,1.193820,2,7,M,lambda,M,x
TTTGTTGAGTAACGAT-1-2,2665,2665,18230.0,674.0,3.697202,2,5,A,kappa,A,lo
TTTGTTGCACGTTGGC-1-2,2211,2211,19481.0,384.0,1.971151,2,5,A,kappa,A,lo


generate pseudobulk expression data per group

type of group definitions:
- 'group': leiden cluster
- 'donor_sec': donor, high/low
- 'sec': high/low

In [34]:
# you can begin by seeing how many cells there are per group

# is there even balance of cells per group?
data.obs['leiden'].value_counts()

0     2497
1     2305
2     2234
3     1251
4      971
5      840
6      817
7      754
8      503
9      388
10     332
Name: leiden, dtype: int64

In [36]:
# look @ cells per group, donor if needed to check for imbalance
# data.obs.groupby(['leiden','sample']).size()

In [6]:
# set random seed for reproducibility
random.seed(21)

calculation of pseudobulk replicates/gene expression aggregation adapted from:

note, i found some typos and corrected them from original script
https://github.com/mousepixels/sanbomics_scripts/blob/main/pseudobulk_pyDeseq2.ipynb

In [7]:
# specify approx how many cells per pseudobulk group?
pb_ncells = 50 

In [21]:
# set which column in metadata you want to make pseudobulk groups within
pb_group = 'leiden'

In [37]:
# this is about many pseudobulk datapoints you will get per group
## below script will automatically skip group if it has <=10 cells, then it will aggregate it into 1 group
data.obs[pb_group].value_counts()/pb_ncells

0     124.85
1     115.25
2     111.70
3      62.55
4      48.55
5      42.00
6      40.85
7      37.70
8      25.15
9      19.40
10     16.60
Name: leiden, dtype: float64

In [39]:
data.layers

Layers with keys: counts

In [38]:
# making sure that the object layer 'counts' is actual counts and not log counts
pd.DataFrame.sparse.from_spmatrix(data.layers['counts']) # you can see that the last column is ADT counts, which is discrete

,0,1,2,3,4,5,6,7,8,9,...,36682,36683,36684,36685,36686,36687,36688,36689,36690,36691
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,258.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0
12888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,276.0
12889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0
12890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0


In [43]:
# make pseudobulk by specific "group"; output the pseudobulk summed counts and converted to CPM

pbs = [] # init a list to append each pseudobulk AnnData obj

## loop through each group, assign random pb subgroups, perform pb sum of counts and cpm
for sample in data.obs[pb_group].unique():
    samp_cell_subset_rna = data[data.obs[pb_group] == sample]
#     samp_cell_subset_adt = g_adt[rna.obs['group'] == sample]
    
    samp_cell_subset_rna.X = samp_cell_subset_rna.layers['counts'] #make sure to use raw data
#     samp_cell_subset_adt.X = samp_cell_subset_adt.layers['counts'] #make sure to use raw data

    
    totcells = samp_cell_subset_rna.shape[0]
    
    # determine how many groups per label
    
    if totcells > 10:
        pbreps =  np.round(totcells/pb_ncells).astype(int)  # get N pseudobulk groups per pb_ncells cells; round number to integer
    else:
        pbreps = 1 # aggregate all cells in 1 group
        
    print(sample, totcells, pbreps) # print group, # tot cells, how many pb groups are formed
    
    # obtain cell indices per pseudobulk group
    indices = list(samp_cell_subset_rna.obs_names) # get all cell indices belonging to this group
    random.shuffle(indices) # randomly shuffle
    indices = np.array_split(np.array(indices), pbreps) #change number here for number of replicates desired; split into random groups
    
    for i, pseudo_rep in enumerate(indices):
        
        # subset object per pb group
        ind_subset = samp_cell_subset_rna[samp_cell_subset_rna.obs.index.isin(pseudo_rep.tolist()),:].copy()
        
        # sum pseudobulk raw counts
        rep_adata = sc.AnnData(X = ind_subset.X.sum(axis = 0),
                               var = samp_cell_subset_rna.var[[]]) # .var[[]] retrieves all the feature names
        
        # sum pseudobulk counts and convert to cpm
        rep_adata.layers['cpm'] = ind_subset.X.sum(axis = 0) * 1e6/ind_subset.X.sum().sum() # add cpm as well
        
        ## extract just igg_adt counts
        rep_adata_adt = rep_adata[:, ['IgG_ADT']].to_df().copy()
        
        rep_adata.obs_names = [sample + '_' + str(i)]
        rep_adata.obs['group'] = sample #samp_cell_subset_rna.obs['group'].iloc[0]
        rep_adata.obs['replicate'] = i
        rep_adata.obs['pb_adt_count'] = rep_adata_adt['IgG_ADT'].values.tolist()
        rep_adata.obs['pb_adt_cpm'] = pd.DataFrame(rep_adata.layers['cpm'])[36691].values[0] #pd.DataFrame(rep_adata.layers['cpm'].toarray())[36691].values.tolist()

        pbs.append(rep_adata)

1 2305 115
3 1251 63
0 2497 125
5 840 42
6 817 41
2 2234 112
8 503 25
4 971 49
10 332 17
7 754 38
9 388 19


In [44]:
# concat all pseudobulk populations together into one AnnData object
pb = sc.concat(pbs)

In [45]:
# the metadata will also have the same metadata and corresponding ADT (SEC-seq secreted IgG) count as well
## each pseudobulk group is defined by indices here (group_pseudobulkgroup) e.g. leidencluster_pseudobulkgroup
pb.obs

,group,replicate,pb_adt_count,pb_adt_cpm
1_0,1,0,2493.0,9232.717819
1_1,1,1,2139.0,7414.674052
1_2,1,2,1822.0,6475.942690
1_3,1,3,3373.0,12061.505453
1_4,1,4,1838.0,7278.026142
...,...,...,...,...
9_14,9,14,859.0,5164.618456
9_15,9,15,1549.0,7354.407422
9_16,9,16,1954.0,11420.155347
9_17,9,17,1108.0,5737.157415


In [46]:
# create df for CPM expression of pseudobulk data for all genes
pb_cpm_rna_df = pd.DataFrame(data=pb.layers['cpm'], index=pb.obs_names, columns=pb.var_names)
pb_cpm_rna_df

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1,...,BARF0,BALF2,BALF1,BARF1,LMP-1,BNLF2b,BNLF2a,EBER1,EBER2,IgG_ADT
1_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9232.717819
1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7414.674052
1_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6475.942690
1_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12061.505453
1_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7278.026142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5164.618456
9_15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7354.407422
9_16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11420.155347
9_17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5737.157415


In [48]:
# get highly variable genes for gene expression data
hv_rna = sc.pp.highly_variable_genes(rna, n_top_genes=4000, flavor='seurat_v3', layer='counts', inplace=False)
hv_rna

,means,variances,highly_variable_rank,variances_norm,highly_variable
MIR1302-2HG,0.000000,0.00000,NaN,0.000000,False
FAM138A,0.000000,0.00000,NaN,0.000000,False
OR4F5,0.000000,0.00000,NaN,0.000000,False
AL627309.1,0.001241,0.00124,NaN,0.919638,False
AL627309.3,0.000000,0.00000,NaN,0.000000,False
...,...,...,...,...,...
LMP-1,0.000000,0.00000,NaN,0.000000,False
BNLF2b,0.000000,0.00000,NaN,0.000000,False
BNLF2a,0.000000,0.00000,NaN,0.000000,False
EBER1,0.000000,0.00000,NaN,0.000000,False


In [52]:
# to use for GSVA, you will want to filter for only the highly variable genes to reduce computational burden

#  get HVGs from gene expression data
hv_genes = hv_rna[hv_rna['highly_variable'] == True].index.tolist()
hv_genes[:10]

['SAMD11',
 'AL645608.7',
 'HES4',
 'ISG15',
 'TNFRSF18',
 'TNFRSF4',
 'ANKRD65',
 'AL139246.5',
 'ACOT7',
 'TNFRSF25']

In [54]:
# filter pseudobulk data for only HVGs
pb_cpm_rna_df_hvg = pb_cpm_rna_df.copy()
pb_cpm_rna_df_hvg = pb_cpm_rna_df[hv_genes]
pb_cpm_rna_df_hvg

,SAMD11,AL645608.7,HES4,ISG15,TNFRSF18,TNFRSF4,ANKRD65,AL139246.5,ACOT7,TNFRSF25,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC240274.1,AC233755.2,AC233755.1,AC136616.2,AC141272.1
1_0,0.000000,0.0,0.000000,1799.880008,0.000000,0.000000,0.0,7.406914,0.000000,0.0,...,125.917531,1873.949144,648.104941,137.027902,3270.152360,0.0,3.703457,3.703457,0.0,0.000000
1_1,0.000000,0.0,6.932842,1362.303367,3.466421,0.000000,0.0,3.466421,0.000000,0.0,...,103.992623,1587.620718,592.757954,152.522514,2915.259878,0.0,100.526203,0.000000,0.0,0.000000
1_2,0.000000,0.0,0.000000,1802.032351,78.194698,3.554304,0.0,0.000000,3.554304,0.0,...,78.194698,1247.560859,451.396664,159.943700,2185.897231,0.0,35.543044,0.000000,0.0,0.000000
1_3,0.000000,0.0,0.000000,2045.413910,3.575898,0.000000,0.0,0.000000,0.000000,0.0,...,85.821563,1319.506526,439.835509,118.004649,2395.851958,0.0,21.455391,0.000000,0.0,0.000000
1_4,0.000000,0.0,0.000000,2724.310112,3.959753,0.000000,0.0,3.959753,0.000000,0.0,...,110.873086,1465.108636,621.681232,209.866913,2300.616534,0.0,23.758518,0.000000,0.0,7.919506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_14,0.000000,0.0,0.000000,4124.479931,0.000000,0.000000,0.0,18.037084,0.000000,0.0,...,48.098891,1112.286862,342.704601,42.086530,1388.855487,0.0,0.000000,6.012361,0.0,0.000000
9_15,4.747842,0.0,0.000000,2578.078263,9.495684,0.000000,0.0,9.495684,0.000000,0.0,...,71.217632,1514.561632,379.827368,56.974105,2174.511684,0.0,0.000000,0.000000,0.0,4.747842
9_16,0.000000,0.0,0.000000,3646.968749,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,93.512019,1087.077223,286.380559,87.667518,1578.015324,0.0,0.000000,5.844501,0.0,0.000000
9_17,0.000000,0.0,0.000000,2775.375789,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,62.135279,1221.993818,409.057252,98.380858,1693.186349,0.0,0.000000,0.000000,0.0,0.000000
